In [403]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from agents import GlobalConsensus, EventGlobalConsensus, EventGlobalConsensusTorch
from models import NN, Dummy
from utils import add_params, scale_params, subtract_params, average_params, sum_params
import torch

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of agents failed: Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/disop/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/disop/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/disop/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/opt/homebrew/Caskroom/miniforge/base/envs/disop/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
  File "/opt/homebrew/Caskroom/miniforge/base/envs/disop/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/opt/homebrew/Caskroom/miniforge/base/e

### Experimenting with SGD and adding parameters

In [404]:
def get_copy(params):
    copy = [torch.zeros(param.shape).copy_(param) for param in params]
    return copy

In [405]:
model1 = NN(2,2,1)

copy = get_copy(model1.parameters())

for param in copy:
    param.data = 2*torch.ones(param.shape)
for param in copy:
    print(param)

print('\n----- model 1 -----')
print('before training')
for param in model1.parameters():
    print(param)

# experiment minimizing l(x) = |x - 2|_2
opt = torch.optim.Adam(model1.parameters(), lr=0.01)
loss = torch.Tensor([np.inf])
prev_loss = torch.Tensor([0])
while np.abs(prev_loss.item() - loss.item()) >= 1e-4:
    prev_loss = loss
    opt.zero_grad()
    loss = 0
    for param, copied in zip(model1.parameters(), copy):
        loss += torch.norm(param-copied.data)
    loss.backward()
    opt.step()

print('\nafter training')
for param in model1.parameters():
    print(param)
print('\ncopied')
for param in copy:
    print(param)

tensor([[2., 2.],
        [2., 2.]], grad_fn=<CopyBackwards>)
tensor([2., 2.], grad_fn=<CopyBackwards>)
tensor([[2., 2.]], grad_fn=<CopyBackwards>)
tensor([2.], grad_fn=<CopyBackwards>)

----- model 1 -----
before training
Parameter containing:
tensor([[ 0.4539,  0.5580],
        [-0.2682,  0.7049]], requires_grad=True)
Parameter containing:
tensor([0.6143, 0.2870], requires_grad=True)
Parameter containing:
tensor([[-0.2235, -0.2562]], requires_grad=True)
Parameter containing:
tensor([0.4634], requires_grad=True)

after training
Parameter containing:
tensor([[1.9999, 1.9992],
        [1.9989, 2.0007]], requires_grad=True)
Parameter containing:
tensor([2.0030, 1.9984], requires_grad=True)
Parameter containing:
tensor([[1.9998, 1.9999]], requires_grad=True)
Parameter containing:
tensor([2.0002], requires_grad=True)

copied
tensor([[2., 2.],
        [2., 2.]], grad_fn=<CopyBackwards>)
tensor([2., 2.], grad_fn=<CopyBackwards>)
tensor([[2., 2.]], grad_fn=<CopyBackwards>)
tensor([2.], grad_f

### Event-Based ADMM with Torch

In [406]:
# Initial lambdas must sum to 0!
delta = 0.01
rho = 1
N = 4

agents = [
    EventGlobalConsensusTorch(
        N=N, 
        rho=rho, 
        model=NN(2,3,1),
        delta=delta
    ) 
    for _ in range(N)
]

for agent in agents:
    agent.primal_avg = average_params([agent.model.parameters() for agent in agents])
    for param in agent.primal_avg:
        print(param)

for agent in agents:
    for param in agent.primal_avg:
        print(param)

print('\nbefore training')
for agent in agents:
    for param in agent.model.parameters(): print(param)

tensor([[-0.2060, -0.1057],
        [ 0.2478, -0.1163],
        [-0.1273, -0.0881]], grad_fn=<DivBackward0>)
tensor([-0.2288,  0.0030, -0.2114], grad_fn=<DivBackward0>)
tensor([[ 0.2460, -0.1896, -0.2742]], grad_fn=<DivBackward0>)
tensor([0.0827], grad_fn=<DivBackward0>)
tensor([[-0.2060, -0.1057],
        [ 0.2478, -0.1163],
        [-0.1273, -0.0881]], grad_fn=<DivBackward0>)
tensor([-0.2288,  0.0030, -0.2114], grad_fn=<DivBackward0>)
tensor([[ 0.2460, -0.1896, -0.2742]], grad_fn=<DivBackward0>)
tensor([0.0827], grad_fn=<DivBackward0>)
tensor([[-0.2060, -0.1057],
        [ 0.2478, -0.1163],
        [-0.1273, -0.0881]], grad_fn=<DivBackward0>)
tensor([-0.2288,  0.0030, -0.2114], grad_fn=<DivBackward0>)
tensor([[ 0.2460, -0.1896, -0.2742]], grad_fn=<DivBackward0>)
tensor([0.0827], grad_fn=<DivBackward0>)
tensor([[-0.2060, -0.1057],
        [ 0.2478, -0.1163],
        [-0.1273, -0.0881]], grad_fn=<DivBackward0>)
tensor([-0.2288,  0.0030, -0.2114], grad_fn=<DivBackward0>)
tensor([[ 0.246

### Run simulation

In [407]:
from tqdm import tqdm
comm = 0

t_max = 200
for t in tqdm(range(t_max)):
    
    # Primal Update
    for agent in agents:
        agent.primal_update()

    # Residual update in the case of communication
    C = []
    for agent in agents:
        if agent.broadcast: 
            comm += 1
            C.append(agent.residual)
    if C:
        # If communicaiton set isn't empty
        residuals = [x for x in sum_params(C)]
        for agent in agents:
            add_params(agent.primal_avg, residuals)

    # Dual update
    for agent in agents:
        agent.dual_update()

load = comm/(t_max*len(agents))
print(f'Communication load = {load}')

print('\nParams')
for agent in agents:
    for param in agent.model.parameters():
        print(param)

print('\nAverage')
for agent in agents:
    for param in agent.primal_avg:
        print(param)

100%|██████████| 200/200 [01:00<00:00,  3.33it/s]

Communication load = 0.91375

Params
Parameter containing:
tensor([[2.0033, 2.0003],
        [1.9958, 2.0068],
        [2.0003, 1.9922]], requires_grad=True)
Parameter containing:
tensor([2.0010, 2.0032, 2.0039], requires_grad=True)
Parameter containing:
tensor([[1.9932, 2.0060, 2.0034]], requires_grad=True)
Parameter containing:
tensor([1.9927], requires_grad=True)
Parameter containing:
tensor([[1.9964, 2.0059],
        [2.0059, 1.9942],
        [1.9932, 2.0027]], requires_grad=True)
Parameter containing:
tensor([1.9979, 1.9982, 1.9989], requires_grad=True)
Parameter containing:
tensor([[2.0112, 2.0016, 1.9929]], requires_grad=True)
Parameter containing:
tensor([1.9940], requires_grad=True)
Parameter containing:
tensor([[2.0030, 1.9902],
        [1.9970, 2.0057],
        [2.0013, 2.0031]], requires_grad=True)
Parameter containing:
tensor([2.0045, 2.0064, 1.9861], requires_grad=True)
Parameter containing:
tensor([[1.9974, 2.0038, 1.9899]], requires_grad=True)
Parameter containing:
tens